In [1]:
import requests as rq
import pandas as pd
import re

In [2]:
apikey = "aopBL0uXOuKPeqdXonnZJ13o9XHaTyBS"

In [3]:
payload = {}
headers = {'apikey': apikey}

base_url = "https://api.apilayer.com/spotify/"
endpoint_url = "album_tracks"
album_url = "album_metadata"

In [4]:
songs = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds','Rating'])

In [5]:
ids = ['7mzrIsaAjnXihW3InKjlC3','7vzYp7FrKnTRoktBYsx9SF','2dqn5yOQWdyGwOpOIi9O4x','4hDok0OAJd57SGIT8xuWJH',
       '5EpMjweRD573ASl7uNiHym','1KlU96Hw9nlvqpBPlSqcTV','6kZ42qRrzov54LcAk4onW9','34OkZVpuzBa9y40DCy0LPR',
       '6DEjYFkNZh67HP7R9PSZvv','1NAmidJlEaVgA3MpcPFYGq','1pzvBxYgT6OVwJLtHkrdQK','6AORtDjduMM3bupSWzbTSG',
       '3lS1y25WAhcqJDATJK70Mq']

In [6]:
for j in range(len(ids)):
    params = {'id': ids[j]}
    r = rq.get(base_url + endpoint_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(r.json())
    data = data["data"].apply(pd.Series)
    data = data["tracks"].apply(pd.Series)
    data = data["items"].apply(pd.Series)
    table = pd.DataFrame(columns=['Title','Album','Track Number','Play Count','Duration in Seconds','Rating'])
    for i in range(data.shape[1]):
        track = data.iloc[0,i]
        table.loc[i,'Title'] = track['track']['name']
        table.loc[i,'Track Number'] = track['track']['trackNumber']
        table.loc[i,'Play Count'] = track['track']['playcount']
        table.loc[i,'Duration in Seconds'] = track['track']['duration']['totalMilliseconds']/1000
        table.loc[i,'Rating'] = track['track']['contentRating']['label']
    ra = rq.get(base_url + album_url, params=params, headers=headers, data=payload)
    data = pd.DataFrame(ra.json())
    table['Album'] = data['data']['album']['name']
    songs = pd.concat([songs,table])

In [7]:
songs['Track Number']=songs['Track Number'].apply(lambda x: pd.to_numeric(x))
songs['Play Count']=songs['Play Count'].apply(lambda x: pd.to_numeric(x))
songs['Duration in Seconds']=songs['Duration in Seconds'].apply(lambda x: pd.to_numeric(x))

In [8]:
songs['Album'] = songs['Album'].str.replace("Taylor's Version",'')
songs['Album'] = songs['Album'].str.replace("Deluxe Edition",'')
songs['Album'] = songs['Album'].str.replace("deluxe version",'')
songs['Album'] = songs['Album'].str.replace("3am Edition",'')
songs['Album'] = songs['Album'].str.split(r'\ \(',expand=True)[0]
songs['Album'] = songs['Album'].str.strip()
songs['Title'] = songs['Title'].str.replace("Taylor",'')
songs['Title'] = songs['Title'].str.replace("s Version",'')
songs['Title'] = songs['Title'].str.replace("\u2019",'')
songs['Title'] = songs['Title'].str.replace("\u0027",'')
songs['Title'] = songs['Title'].str.replace('From The Vault','')
songs['Title'] = songs['Title'].str.split(r'\ \(',expand=True)[0]
songs['Title'] = songs['Title'].str.strip()

In [9]:
songs_final = songs.drop_duplicates(subset=['Title'], keep='first')
songs_final = songs_final.drop(['Play Count','Duration in Seconds'], axis=1)
songs_final['Play Count'] = list(songs.groupby([songs['Title']]).sum()['Play Count'])
songs_final['Duration in Seconds'] = list(songs.groupby([songs['Title']]).mean()['Duration in Seconds'])
songs_final.reset_index(inplace=True)
songs_final = songs_final.drop('index', axis=1)

In [10]:
songs_final

,Title,Album,Track Number,Rating,Play Count,Duration in Seconds
0,Tim McGraw,Taylor Swift,1,NONE,447861257,208.1860
1,Picture To Burn,Taylor Swift,2,NONE,421459211,230.5465
2,Teardrops On My Guitar - Radio Single Remix,Taylor Swift,3,NONE,14811611,220.1460
3,A Place in this World,Taylor Swift,4,NONE,24225943,199.2000
4,Cold As You,Taylor Swift,5,NONE,200549043,223.2930
...,...,...,...,...,...,...
199,Paris,Midnights,16,NONE,214410565,231.0000
200,High Infidelity,Midnights,17,NONE,211497418,195.1060
201,Glitch,Midnights,18,NONE,147192496,245.4400
202,"Wouldve, Couldve, Shouldve",Midnights,19,NONE,542373719,214.7060


In [11]:
songs_final.to_csv('TaylorSwiftSongs.csv')